## AIPI 590 - XAI | Assignment #3
### Interpretable ML
#### Author: Tal Erez
#### Colab Notebook:
[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/notthattal/AdversarialAI_Patch/blob/main/adversarial_patch.ipynb#scrollTo=jv7U4G4FodQf)

### How to Run in Colab

1. Verify you are running on a GPU. On the top right of the screen click the down arrow in between "RAM/Disk" and "Gemini"  -> Change Runtime Type -> T4 GPU -> Save
2. Ensure Environment Variables are set/Set Environment Variables. On the left-hand side of the screen click the key -> Add new secret -> set the following environment variables:
    - CHECKPOINT_PATH: /content/AdversarialAI_Patch/saved_models/
    - DATASET_PATH: /content/AdversarialAI_Patch/data/
    - IMAGE_PATH: /content/AdversarialAI_Patch/saved_models/images/
    - TORCH_HOME: /content/AdversarialAI_Patch/saved_models/
3. For each environment variable make sure notebook access is activated (a switch located to the left-hand side of each individual environment variable)

### Introduction

This notebook uses a [Telco Customer Churn](https://www.kaggle.com/datasets/blastchar/telco-customer-churn/code) database and attempts to compare predicting the likelihood of customer churn using linear regression, logistic regression and a logistic general additive model

### Install required dependencies and import required packages

In [2]:
import os

# Remove Colab default sample_data if it exists
if os.path.exists("./sample_data"):
    !rm -r ./sample_data

# Clone GitHub files to colab workspace
repo_name = "InterpretableML_I"

# Check if the repo already exists
if not os.path.exists("/content/" + repo_name):
    git_path = 'https://github.com/notthattal/InterpretableML_I.git'
    !git clone "{git_path}"
else:
    print(f"{repo_name} already exists.")

# Change working directory to location of notebook
path_to_notebook = os.path.join("/content/" + repo_name)
%cd "{path_to_notebook}"
%ls

#Install the requirements for this package
!pip install -r requirements.txt

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pygam import LogisticGAM
import seaborn as sns
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, f1_score
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.stattools import durbin_watson
from statsmodels.stats.diagnostic import het_breuschpagan

### Preparing the Dataframe
1. Creates a dataframe from the CSV
2. Cleans the dataframe by dropping all rows with empty strings, whitespace-only strings and NaN values
3. Converts all the "Yes/No" categorical columns to binary 1 or 0
4. One-hot encodes the non-binary categorical columns and drops duplicate one-hot encoded columns (e.g. 'OnlineBackup_No internet service and 'DeviceProtection_No internet service' would just become "No Internet Service")
5. Remove the Customer ID column from the dataframe as this feature should not affect the outcome
6. Convert the boolean columns from the one-hot-encoding to integers and the totalCharges column to a float instead of an object
7. Scales the columns containing continuous data
8. Assign the feature columns to X and the target column to y

In [64]:
#read the csv into the dataframe
df = pd.read_csv("./data/WA_Fn-UseC_-Telco-Customer-Churn.csv")

# Strip any whitespace from the column (important for cases with invisible spaces)
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

# Replace all empty strings and whitespace-only strings with NaN
df = df.replace(r'^\s*$', np.nan, regex=True)

# Drop rows with any NaN or empty string
df = df.dropna()

# Convert all binary columns to integers False = 0 and True = 1 
yes_no_cols = ["Partner", "Dependents", "PhoneService", "PaperlessBilling", "Churn"]
for i in yes_no_cols:
    df[i] = df[i].map({'No': 0, 'Yes': 1})
df["gender"] = df["gender"].map({'Male': 0, 'Female': 1})

#One hot encode dataframe
df = pd.get_dummies(df, columns=["MultipleLines", "OnlineSecurity", "OnlineBackup", 
                                 "DeviceProtection", "TechSupport", "StreamingTV", 
                                 "StreamingMovies", "InternetService", "Contract", 
                                 "PaymentMethod"], drop_first=True)

#Drop columns that are essentially duplicates of another column in the df
df = df.drop(columns=['MultipleLines_No phone service', 'OnlineSecurity_No internet service', 'OnlineBackup_No internet service', 'DeviceProtection_No internet service', 'TechSupport_No internet service', 'StreamingTV_No internet service', 'StreamingMovies_No internet service'])

#remove the ID column as it we will not need it for regression
cleaned_df = df.drop(columns=['customerID'])

#convert the TotalCharges column from string to float
cleaned_df['TotalCharges'] = df['TotalCharges'].astype(float)

#convert all boolean columns to integer
for col in cleaned_df.columns:
    if cleaned_df[col].dtype == 'bool':  # Check for boolean dtype
        cleaned_df[col] = cleaned_df[col].astype(int)


scale_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']
scaler = StandardScaler()

# scale the columns containing continuous data
cleaned_df[scale_cols] = scaler.fit_transform(cleaned_df[scale_cols])

y = cleaned_df["Churn"]
X = cleaned_df.drop(columns=["Churn"])

### Assessing Linearity
According to this [source](https://bookdown.org/rwnahhas/RMPH/mlr-linearity.html), "For a binary predictor, the linearity assumption is always true – there are two means (the mean outcome at each level of the predictor) and a straight line always perfectly fits two points." Thus, for our case in regards to churn, the Linearity assumption has been met.

### Assessing Independence

Independence of observations for this dataset is assumed due to each row consisting of an individual user and one user's plan shouldn't influence another's plan.

### Assessing MultiCollinearity

In [ ]:
# Calculate correlation matrix
corr_matrix = X.corr()

#create the figure
plt.figure(figsize=(14, 12)) 

# Create a heatmap using seaborn
sns.heatmap(corr_matrix, annot=True, fmt=".2f", annot_kws={"size": 8}, cmap='coolwarm', center=0, linewidths=0.5)

# Add labels and title
plt.title('Correlation Matrix Heatmap')
plt.show()

### Visualizing MultiCollinearity using VIF ([source](https://www.geeksforgeeks.org/detecting-multicollinearity-with-vif-python/))

For deciding on a threshold for which to conclude multicollinearity isn't an issue using VIF, there are a number of different sources that recommend a different threshold. After peering through a few different ones, the most common threshold I saw was 0-5 for no multicollinearity, 5-10 for moderate collinearity and 10+ for high multicollinearity. I chose to use a threshold of 5.

In [ ]:
# Create a DataFrame to store VIF values
vif = pd.DataFrame()
vif["Feature"] = X.columns

#calculate the VIF scores and store them in a column in the vif df
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

# Create the bar chart to visualize VIF
plt.figure(figsize=(10, 8))
plt.barh(vif['Feature'], vif['VIF'], color='skyblue')
plt.xlabel('VIF Value')
plt.ylabel('Features')
plt.title('Variance Inflation Factors for Features')
plt.show()


### As we can see, MonthlyCharges has a very high multicollinearity score. let's visualize it in another way

In [ ]:
correlation_matrix = X.corr()
corr_with_monthly = correlation_matrix['MonthlyCharges'].drop('MonthlyCharges')  # Drop 'MonthlyCharges' correlation with itself

# Step 3: Create the bar plot
plt.figure(figsize=(10, 6))
plt.barh(corr_with_monthly.index, corr_with_monthly.values, color=['#1f77b4' if x > 0 else '#d62728' for x in corr_with_monthly])

# Step 4: Add labels and title
plt.xlabel('Correlation with MonthlyCharges')
plt.title('Correlation of Features with MonthlyCharges')

# Step 5: Display the plot
plt.show()

### What to do about the monthly charge problem?
This problem is actually two-fold. First, after dropping the monthly charge column and re-evaluating the VIF, I found the total charges and tenure were still above the threshold of 10 we set. This makes sense, as the longer a person stays with the company the higher the total charges would be. In order to meet our threshold, I chose to drop the monthly charges and tenure columns. Monthly charges can be correlated to many other features such as if the customer pays for fiber optics, phone service etc. Once we drop monthly charges and tenure, we still have PhoneService which surpasses our threshold, so I chose to drop all three columns to meet the threshold we decided on earlier

In [ ]:
#Drop the monthly charges, tenure columns and PhoneService columns from the training data
X = X.drop(columns=["MonthlyCharges", 'tenure', 'PhoneService'])

#Recalculate the VIF for the remaining features
vif = pd.DataFrame()
vif["Feature"] = X.columns
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
print(vif)

Great! We are under our threshold.


### Fit the linear regression model and get residuals

In [89]:
#Split the data into a training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

#fit the model
model = LinearRegression().fit(X_train, y_train)

#predict on the test set
y_pred = model.predict(X_test)

#calculate the residuals
residuals = y_test - y_pred

### Assessing Homoscedasticity

To assess for homoscedasticity of residuals I used the Breusch–Pagan test. The result below shows that there is overwhelming evidence of heteroscedasticity in the model, which is expected due to the binary nature of the target

In [ ]:
# Add constant to the model
X_test_with_const = sm.add_constant(X_test)

# Run the Breusch-Pagan test
test = het_breuschpagan(residuals, X_test_with_const)

# Retrieve and report the p-value
p_value = test[1]
print(f'Breusch-Pagan p-value: {p_value}')

### Assessing Normality

To assess normality I used a Q-Q plot to plot the residuals and, as we can see below, the normality assumption was violated in the case of linear regression

In [ ]:
# Q-Q plot for residuals
sm.qqplot(residuals, line='45')
plt.title('Q-Q Plot of Residuals')
plt.show()

### Assessing Autocorrelation

Autocorrelation isn't expected as the training data doesn't contain any specific ordering or time-dependent features, but to mathematically verify this assumption I chose to use the Durbin-Watson test as recommended by this [source](https://godatadrive.com/blog/basic-guide-to-test-assumptions-of-linear-regression-in-r). As we can see, the statistical result is very close to 2 indicating no autocorrelation of the residuals 

In [ ]:
# Run and report the results of the durbin_watson test
dw_results = durbin_watson(residuals)
print(f'Durbin-Watson statistic: {dw_results}')

In [ ]:
# Calculate and report the MSE and R-Squared metrics
mse = mean_squared_error(y_test, y_pred)
r_squared = r2_score(y_test, y_pred)

print(f'Mean Squared Error: {mse}')
print(f'r-squared: {r_squared}')

# Get and print the intercept and coefficients
print(f'Intercept: {model.intercept_}')
coefficients = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': model.coef_
})
print(coefficients)

Intuitively we can tell by the values of the coefficients that this model did not perform well in determining Churn. Having a negative relationship with TotalCharges implies that as the company increases the total cost of the service, the customer is less likely to churn. As much as the company would love that, it doesn't seem very realistic.

In terms of model performance, we know that our values can range between 0 and 1 and thus if we calculate the RMSE using the Mean-Squared Error of approximately 0.145 (RMSE = √MSE = √0.145) we can see the model's prediction on average is ≈ 0.381 off from the actual value. Combined with an R-Squared that only explains about 25.8% of the variance in the model, this leads us to the conclusion that the model is underfitting the data.

## Logistic Regression

For Logistic Regression, we have already assed our data for linearity, multicollinearity and independence of observations. For meeting the requirement of a large sample size, there isn't much we can do as these are the customer data we have, unless this is only a portion of our customer base. Then, perhaps using a larger dataset that encompasses more of our customers would be acceptable. 

In [ ]:
#Fit the Logistic Regression model
model = LogisticRegression(max_iter=500).fit(X_train, y_train)

#Predict the model on the test set
y_pred = model.predict(X_test)

#Calculate and report the accuracy and f1 scores
accuracy = accuracy_score(y_test, y_pred)
f1_val = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'f1-score: {f1_val}')

# Get and print the intercept and coefficients
print(f'Intercept: {model.intercept_}')
coefficients = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': model.coef_.flatten()
})
print(coefficients)

We can see that the accuracy of this model is close to 80% which doesn't seem too bad. However our f1 score still indicates that this model is underfitting the data and needs improvement

### Logistic GAM

For GAM, again we have already met the assumptions for linearity, multicollinearity, sample size and know that we have violated homoscedasticity. The results below show almost identical performance to Logistic Regression and thus we can reach the same conclusion.


In [ ]:
#Create and fit the GAM to the training set
gam = LogisticGAM() 
gam.fit(X_train, y_train)

# Make predictions on the test set
y_pred_prob = gam.predict_proba(X_test)
y_pred = (y_pred_prob >= 0.5).astype(int)

#Calculate and report the accuracy and f1 scores
accuracy = accuracy_score(y_test, y_pred)
f1_val = f1_score(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print(f'f1-score: {f1_val}')

# Get and print the intercept and coefficients
print(f'Intercept: {model.intercept_}')
coefficients = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': model.coef_.flatten()
})
print(coefficients)

### Discussion

While linear regression was easy to interpret, we can see based on the metrics calculated that the performance of the model was not ideal. This should be expected as the actual data is binary, and linear regression is meant for continuous variables. For both logistic regression and the GAM we saw similar results in terms of accuracy and f1 score, and were able to interpret that both models are underfitting the data. Given that the GAM took longer to run to retrieve the same results as the logistic regression model, if we follow Occam's Razor, we should select logistic regression as the model chosen to predict churn. 

### Citations:

https://www.kaggle.com/datasets/blastchar/telco-customer-churn/code

https://www.geeksforgeeks.org/detecting-multicollinearity-with-vif-python/

https://godatadrive.com/blog/basic-guide-to-test-assumptions-of-linear-regression-in-r

https://en.wikipedia.org/wiki/Homoscedasticity_and_heteroscedasticity#Testing

https://bookdown.org/rwnahhas/RMPH/mlr-linearity.html